In [1]:
import os

In [2]:
%pwd

'/home/ziro/Desktop/Isham/Personal/text-summarizer-project/text-summarizer-project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/ziro/Desktop/Isham/Personal/text-summarizer-project/text-summarizer-project'

In [5]:
from pydantic import BaseModel
from pathlib import Path

class DataIngestionConfig(BaseModel):
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from utils import *
from textSummarizer import *

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [8]:
config_file_path: Path = CONFIG_FILE_PATH
config = read_yaml(config_file_path)

[2023-06-04 00:36:08,056: INFO: helper: yaml file: config/config.yaml loaded successfully]


In [9]:
config.artifacts_root

'databox'

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-04 00:40:32,011: INFO: helper: yaml file: config/config.yaml loaded successfully]
[2023-06-04 00:40:32,013: INFO: helper: yaml file: config/params.yaml loaded successfully]
[2023-06-04 00:40:32,014: INFO: helper: created directory at: databox]
[2023-06-04 00:40:32,014: INFO: helper: created directory at: databox/data_ingestion]
[2023-06-04 00:40:33,371: INFO: 1434958058: databox/data_ingestion/summarizer-data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C948:4820:15B7DF:1F73A9:647BA540
Accept-Ranges: bytes
Date: Sat, 03 Jun 2023 20:40:33 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990029-FJR